##  Import modules
- **pandas** and numpy for data manipulation
- **turicreate** for performing model selection and evaluation
- **sklearn** for splitting the data into train and test set

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import turicreate as tc
%matplotlib inline


## Load Data

In [43]:
customers_df = pd.read_csv('recommend_1.csv')
transactions_df = pd.read_csv('trx_data.csv')

In [44]:
print(customers_df.shape)
customers_df.head()

(1000, 1)


,customerId
0,1553
1,20400
2,19750
3,6334
4,27773


In [45]:
print(transactions_df.shape)
transactions_df.head()

(62483, 2)


,customerId,products
0,0,20
1,1,2|2|23|68|68|111|29|86|107|152
2,2,111|107|29|11|11|11|33|23
3,3,164|227
4,5,2|2


## Create data with user, item, and target field

In [46]:
transactions_df['products'] = transactions_df['products'].apply(lambda x: [int(i) for i in x.split('|')])

In [47]:
transactions_df.head()

,customerId,products
0,0,[20]
1,1,"[2, 2, 23, 68, 68, 111, 29, 86, 107, 152]"
2,2,"[111, 107, 29, 11, 11, 11, 33, 23]"
3,3,"[164, 227]"
4,5,"[2, 2]"


In [48]:
#Data Preperation
data = pd.melt(transactions_df.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products')\
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.int64)
data.head()

,customerId,productId,purchase_count
0,0,1,2
1,0,13,1
2,0,19,3
3,0,20,1
4,0,31,2


In [49]:
data.shape

(133585, 3)

In [50]:
data.head()

,customerId,productId,purchase_count
0,0,1,2
1,0,13,1
2,0,19,3
3,0,20,1
4,0,31,2


## Train test split

In [60]:
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)



(106868, 3) (26717, 3)


In [61]:
# Using turicreate library, we convert dataframe to SFrame - this will be useful in the modeling part

train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

In [62]:
train_data.head()

customerId,productId,purchase_count
2488,5,1
8413,29,2
3336,196,1
2478,21,1
21799,135,1
19565,206,4
13155,14,1
8096,4,3
4712,10,3
4685,235,2


In [63]:
test_data.head()

customerId,productId,purchase_count
25635,297,1
13169,218,1
1078,6,2
4862,41,3
19824,125,1
19492,121,2
10313,1,2
604,174,1
5117,285,1
18675,147,3


In [55]:
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

## Popularity model

In [54]:
# constant variables to define field names include:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(customers_df[user_id])
n_rec = 10           # number of items to recommend
n_display = 30       # to display the first few rows in an output dataset

In [67]:
popularity_model = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23335 users and 300 items.

Data prepared in: 0.115462s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.018234s

recommendations finished on 1000/1000 queries. users per second: 119417

## Colaborate filtering

In [68]:
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23335 users and 300 items.

Data prepared in: 0.104193s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.024468s

recommendations finished on 1000/1000 queries. users per second: 195274

## Pearson similarity of Collaborative filtering

In [69]:
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23335 users and 300 items.

Data prepared in: 0.103618s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.022599s

recommendations finished on 1000/1000 queries. users per second: 148699

## Model Evaluation

#### i. RMSE (Root Mean Squared Errors)
-- Measures the error of predicted values
-- Lesser the RMSE value, better the recommendations

#### ii. Recall
-- What percentage of products that a user buys are actually recommended?
-- If a customer buys 5 products and the recommendation decided to show 3 of them, then the recall is 0.6

#### iii. Precision
-- Out of all the recommended items, how many the user actually liked?
-- If 5 products were recommended to the customer out of which he buys 4 of them, then precision is 0.8

### So our aim has to be optimizing both recall and precision (to be close to 1 as possible).

In [70]:
models_w_counts = [popularity_model, cos, pear]
names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']

Lets compare all the models we have built based on RMSE and precision-recall characteristics:

In [71]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/13856 queries. users per second: 165865

recommendations finished on 2000/13856 queries. users per second: 194571

recommendations finished on 3000/13856 queries. users per second: 194793

recommendations finished on 4000/13856 queries. users per second: 200361

...

recommendations finished on 10000/13856 queries. users per second: 194609

recommendations finished on 11000/13856 queries. users per second: 194103

recommendations finished on 12000/13856 queries. users per second: 194963

recommendations finished on 13000/13856 queries. users per second: 196514


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    | 0.0013712471131639718 | 0.0008371824480369508 |
|   2    |  0.00335594688221709  | 0.0034656304223639705 |
|   3    |  0.004041570438799083 | 0.0064605417663659605 |
|   4    |  0.007379474595842952 |  0.016260577831600346 |
|   5    |  0.006177829099307199 |  0.017015708838968704 |
|   6    |  0.006507409545804479 |  0.021129424142782086 |
|   7    |  0.005887083470801724 |  0.022302201279040743 |
|   8    |  0.005602266166281751 |  0.024580663343876936 |
|   9    |  0.005404798562997241 |  0.026792185943679013 |
|   10   | 0.0053045612009237995 |  0.029484246457260877 |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.0369648458587792

Per User RMSE (best)
+------------+------------------------+-----

## Final Output

In [72]:
final_model = tc.item_similarity_recommender.create(tc.SFrame(data), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_count', similarity_type='cosine')
recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Preparing data set.

Data has 133585 observations with 24429 users and 300 items.

Data prepared in: 0.121831s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.02834s

recommendations finished on 1000/1000 queries. users per second: 75591.5

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     | 0.14465173482894897  |  1   |
|    1553    |     5     | 0.09291791915893555  |  2   |
|    1553    |     1     | 0.09233839511871338  |  3   |
|    1553    |     35    | 0.08991665840148926  |  4   |
|    1553    |     17    | 0.07552688121795655  |  5   |
|    1553    |     33    |  0.0724305510520935  |  6   |
|    1553    |     61    | 0.06554533243179321  |  7   |
|    1553    |     21    | 0.056928062438964845 |  8   |
|    1553    |    167    |  0.0536272406578064  |  9   |
|    1553    |     15    | 0.05270164012908936  |  10  |
|   20400    |    280    | 0.07982701063156128  |  1   |
|   20400    |     1     | 0.05004173517227173  |  2   |
|   20400    |    122    | 0.04513084888458252  |  3   |
|   20400    |     6     | 0.043053507804870605 |  4   |
|   20400    |    160    | 0.04

## Output DataFrame

In [73]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()

(10000, 4)


,customerId,productId,score,rank
0,1553,2,0.144652,1
1,1553,5,0.092918,2
2,1553,1,0.092338,3
3,1553,35,0.089917,4
4,1553,17,0.075527,5
